In [ ]:
import json
import pandas as pd
from src.analysis import get_statistics,run_individual_model_analysis
departments=["nephrology department",
             "gynecology department",
            "endocrinology department", 
            "neurology department",
             "pediatrics department",
             "cardiac surgical department",                          
             "gastrointestinal surgical department",
             "respiratory medicine department",
             "gastroenterology department",
             "urinary surgical department",
             "hepatobiliary and pancreas surgical department",
             "hematology department"
            ]

In [2]:
types=["single_shot","with_reasons","check_others_input"]
for type in types:
    get_statistics(type)

### improvement analysis

In [7]:
import json
import pandas as pd
departments=["nephrology department",
             "gynecology department",
            "endocrinology department", 
            "neurology department",
             "pediatrics department",
             "cardiac surgical department",                          
             "gastrointestinal surgical department",
             "respiratory medicine department",
             "gastroenterology department",
             "urinary surgical department",
             "hepatobiliary and pancreas surgical department",
             "hematology department"
            ]
models=["gpt-4o","llama3.1","gemma2"]

with open(f"results/analysis/with_reasons.json", "r") as file:
    first_dict = json.load(file)
with open(f"results/analysis/check_others_input.json", "r") as file:
    second_dict = json.load(file)
all_dict={}
for department in departments:
    all_dict[department]={}
    first_department=first_dict[department]
    second_department=second_dict[department]
    correct=[]
    false=[]
    all=first_department[models[0]]["true"]+first_department[models[0]]["false"]
    all.sort()
    for model in models:
        first_correct=first_department[model]["true"]
        second_correct=second_department[model]["true"]
        both_fail=0
        improve=0
        second_fail=0
        both_correct=0
        for key in all:
            if key in first_correct:
                if key in second_correct:
                    both_correct+=1
                else:
                    second_fail+=1
            elif key in second_correct:
                improve+=1
            else:
                both_fail+=1
        all_dict[department][model]={"both_fail":both_fail,"both_correct":both_correct,
                                     "improved":improve,"second_failed":second_fail}
for model in models:
    model_results = []
    for department in departments:  
        results = all_dict[department][model]
        results["department"] = department
        model_results.append(results)
    
    # Create DataFrame
    df = pd.DataFrame(model_results)
    
    # Set 'department' as the index
    df.set_index('department', inplace=True)
    
    # Add a row with the sum of all columns
    df.loc['Total'] = df.sum(numeric_only=True)  # Ensure it sums only numeric columns
    
    # Save to CSV
    df.to_csv(f"results/analysis/improvement_{model}.csv")

            

### 2 model analysis

In [7]:

types = ["check_others_input","check_others_input_without_mine"]
models = ["llama3.1", "gemma2"]
folder="results-experiments/2_model_experiments"
run_individual_model_analysis(models,types,folder)

### deepseek and qwen3

In [8]:
types = ["single_shot"]
models = ["deepseek-r1", "qwen3"]
folder="results-experiments/qwen-deep-seek"
run_individual_model_analysis(models,types,folder)

### self-refinement

In [9]:
types = ["self_refinement"]
models = ["llama3.1", "gemma2","mistral-nemo","gpt-4o"]
folder="results-self-refinement"
run_individual_model_analysis(models,types,folder)

### gpt-llama-gemma

In [11]:
types = ["single_shot","with_reasons","check_others_input","check_others_input_without_mine"]
models = ["llama3.1", "gemma2","gpt-4o"]
folder="results-gpt-llama-gemma"
run_individual_model_analysis(models,types,folder)